In [1]:
source('/hpc/users/torred23/pipelines/support/init.R')

In [9]:
aggregate_counts <- function(infiles, method, txOut=TRUE, genome='hg38', name_func=function(x) { x_split <- strsplit(x, '/')[[1]]; x_name <- x_split[length(x_split)-1]; return(x_name); }, star_col=2) {

    # Add names
    names(infiles) <- sapply(infiles, name_func)

    # STAR
    if (method == 'star') {
         
        # Read and concatenate
        concatenated_dataframe <- do.call('rbind', sapply(names(infiles), function(x) {df <- as.data.frame(data.table::fread(infiles[x])); df <- df[grepl('ENSG*', df$V1),]; colnames(df)[1] <- 'ensembl_gene_id'; colnames(df)[star_col] <- 'counts'; df$sample <- x; df}, simplify=FALSE))
        
        # Cast
        result <- reshape2::dcast(ensembl_gene_id ~ sample, data = concatenated_dataframe, value.var='counts', fun.aggregate=sum)
               
    # Kallisto and Salmon
    } else if (method %in% c('kallisto', 'salmon')) {
      
        # Get tx2g
        tx2g <- data.table::fread(Sys.glob(glue::glue('/sc/hydra/projects/GuccioneLab/genome-indices/{genome}/ensembl/*-biomart.txt')))

        # Tximport
        result <- tximport::tximport(infiles, type=method, txOut=txOut, tx2g=tx2g[,c('Transcript stable ID version', 'Gene stable ID')])

    }
    
    # Return
    return(result)
        
}

In [10]:
infiles <- Sys.glob('/hpc/users/torred23/pipelines/GuccioneLab/covid-tcell/hydra/s2-alignment.dir/star/*/*out.tab')
star_dataframe <- aggregate_counts(infiles, method='star', genome='hg38')
head(star_dataframe)

,ensembl_gene_id,1A_S1_R1_001,1B_S2_R1_001,1C_S3_R1_001,1E_S4_R1_001,1F_S5_R1_001,1G_S6_R1_001,3A_S7_R1_001,3B_S8_R1_001,3C_S9_R1_001,⋯,5C_S15_R1_001,5D_S16_R1_001,5E_S17_R1_001,5G_S18_R1_001,7A_S19_R1_001,7B_S20_R1_001,7C_S21_R1_001,7D_S22_R1_001,7E_S23_R1_001,7F_S24_R1_001
,<chr>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,⋯,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>
1,ENSG00000000003,42,42,26,22,38,16,30,18,28,⋯,16,14,10,24,16,12,24,18,38,24
2,ENSG00000000005,8,0,2,0,0,0,0,0,0,⋯,2,0,0,0,0,0,0,0,0,0
3,ENSG00000000419,1622,1542,1042,1530,1494,1132,1686,1808,1706,⋯,1456,1270,1120,1302,1376,1208,1472,1710,1574,1596
4,ENSG00000000457,816,904,596,886,780,758,634,422,802,⋯,710,594,524,678,598,560,612,750,668,848
5,ENSG00000000460,164,190,140,162,140,102,110,98,162,⋯,144,114,112,150,262,132,178,232,190,248
6,ENSG00000000938,21454,29264,15542,31384,24234,25276,26542,24120,30468,⋯,22944,20328,20232,22030,17380,18940,21722,26046,29584,26898


In [ ]:
infiles <- Sys.glob('/hpc/users/torred23/pipelines/GuccioneLab/covid-tcell/hydra/s2-alignment.dir/kallisto/*/abundance.h5')
kallisto_txi <- aggregate_counts(infiles, method='kallisto', txOut=FALSE, genome='hg38')
head(kallisto_txi$counts)

1 
2 
3 
4 
5 
6 
7 
8 
9 
10 
11 
12 
13 
14 
15 
16 
17 
18 
19 
20 
21 
22 
23 
24 




In [13]:
infiles <- Sys.glob('/hpc/users/torred23/pipelines/GuccioneLab/covid-tcell/hydra/s2-alignment.dir/salmon/*/quant.sf')
salmon_txi <- aggregate_counts(infiles, method='salmon', txOut=Fgenome='hg38')
head(salmon_txi$counts)

reading in files with read_tsv

1 
2 
3 
4 
5 
6 
7 
8 
9 
10 
11 
12 
13 
14 
15 
16 
17 
18 
19 
20 
21 
22 
23 
24 




,1A_S1_R1_001,1B_S2_R1_001,1C_S3_R1_001,1E_S4_R1_001,1F_S5_R1_001,1G_S6_R1_001,3A_S7_R1_001,3B_S8_R1_001,3C_S9_R1_001,3D_S10_R1_001,⋯,5C_S15_R1_001,5D_S16_R1_001,5E_S17_R1_001,5G_S18_R1_001,7A_S19_R1_001,7B_S20_R1_001,7C_S21_R1_001,7D_S22_R1_001,7E_S23_R1_001,7F_S24_R1_001
ENST00000434970.2,0,0,0,0,0,0,0,0,0,0,⋯,0,0,0,0,0,0,0,0,0,0
ENST00000415118.1,0,0,0,0,0,0,0,0,0,0,⋯,0,0,0,0,0,0,0,0,0,0
ENST00000448914.1,0,0,0,0,0,0,0,0,0,0,⋯,0,0,0,0,0,0,0,0,0,0
ENST00000631435.1,0,0,0,0,0,0,0,0,0,0,⋯,0,0,0,0,0,0,0,0,0,0
ENST00000390583.1,0,0,0,0,0,0,0,0,0,0,⋯,0,0,0,0,0,0,0,0,0,0
ENST00000431440.2,0,0,0,0,0,0,0,0,0,0,⋯,0,0,0,0,0,0,0,0,0,0


In [31]:
# Get files
infiles <- Sys.glob(file.path(results_dir, '*', '*abundance.h5'))

# Add names
names(infiles) <- sapply(infiles, name_func)

In [33]:
# Import
require(tximport)

In [36]:
genome='hg38'

In [42]:
# Get tx2g
tx2g <- data.table::fread(Sys.glob(glue::glue('/sc/hydra/projects/GuccioneLab/genome-indices/{genome}/ensembl/*-biomart.txt')))[,c(1, 4)]

In [53]:
head(txi$counts)

,1A_S1_R1_001,1B_S2_R1_001,1C_S3_R1_001,1E_S4_R1_001,1F_S5_R1_001,1G_S6_R1_001,3A_S7_R1_001,3B_S8_R1_001,3C_S9_R1_001,3D_S10_R1_001,⋯,5C_S15_R1_001,5D_S16_R1_001,5E_S17_R1_001,5G_S18_R1_001,7A_S19_R1_001,7B_S20_R1_001,7C_S21_R1_001,7D_S22_R1_001,7E_S23_R1_001,7F_S24_R1_001
ENSG00000000003,51.05216,40.79417,32.23023,33.15891,44.46954,28.42961,23.3118,32.25710,27.76952,33.67071,⋯,25.53284,16.51529,12.08209,23.44132,18.27322,14.45599,17.61952,22.95925,37.04031,35.04609
ENSG00000000005,2.00000,0.00000,1.00000,0.00000,0.00000,0.00000,0.0000,0.00000,0.00000,1.00000,⋯,1.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000
ENSG00000000419,750.00000,727.00000,496.00000,721.00000,705.00000,542.00000,787.0000,853.00000,814.00000,697.00000,⋯,683.00000,566.00000,523.00000,627.00000,646.00000,561.00000,702.00000,826.00000,730.00000,755.00000
ENSG00000000457,603.07870,646.72254,448.09453,682.70977,558.08977,519.49518,521.2949,431.25536,593.09671,780.54047,⋯,491.88729,469.26460,381.83138,522.13590,474.65664,436.29466,457.43441,590.91226,538.29130,637.10988
ENSG00000000460,145.13014,135.47068,87.20398,119.15808,117.41435,92.39445,89.9817,98.06563,107.90646,134.14691,⋯,98.03182,79.98037,75.98264,95.93890,151.43542,84.70761,132.63278,164.68933,123.11935,161.07990
ENSG00000000938,10498.00000,14261.00000,7633.00000,15374.00000,11941.00000,12390.00000,12951.0000,11913.00000,14857.00000,10971.00000,⋯,11255.00000,9963.00000,9904.00000,10703.00000,8539.00000,9219.00000,10615.00000,12813.00000,14505.00000,13227.00000


In [54]:
head(txi$abundance)

,1A_S1_R1_001,1B_S2_R1_001,1C_S3_R1_001,1E_S4_R1_001,1F_S5_R1_001,1G_S6_R1_001,3A_S7_R1_001,3B_S8_R1_001,3C_S9_R1_001,3D_S10_R1_001,⋯,5C_S15_R1_001,5D_S16_R1_001,5E_S17_R1_001,5G_S18_R1_001,7A_S19_R1_001,7B_S20_R1_001,7C_S21_R1_001,7D_S22_R1_001,7E_S23_R1_001,7F_S24_R1_001
ENSG00000000003,1.462271,1.008560,1.1787432,0.9351069,1.365565,0.8463663,0.5747579,0.7650615,0.7822711,0.9637599,⋯,0.7700359,0.6118139,0.4458162,1.170312,0.5818859,0.5045505,0.5040114,0.5271542,1.061550,0.8454969
ENSG00000000005,0.204246,0.000000,0.1297491,0.0000000,0.000000,0.0000000,0.0000000,0.0000000,0.0000000,0.1015466,⋯,0.1069942,0.0000000,0.0000000,0.000000,0.0000000,0.0000000,0.0000000,0.0000000,0.000000,0.0000000
ENSG00000000419,88.474815,73.089350,73.6780693,71.0112082,76.143561,65.6890103,82.0732153,81.6783605,82.7851247,80.7660621,⋯,84.5059962,86.3125247,77.6555920,81.361865,83.1550146,80.4005526,81.1008401,76.7301620,75.026189,72.6233950
ENSG00000000457,16.622367,14.598551,16.0649069,14.1482122,14.320505,15.4240920,11.0171953,6.5500682,14.2693871,23.0335976,⋯,14.0641424,14.7664669,13.8052737,14.925791,12.8905529,14.3822365,13.3374010,11.8354678,12.181736,14.8341505
ENSG00000000460,4.865771,5.134641,5.3587162,3.9200597,4.933816,2.6953187,2.7450779,2.8922840,4.2197928,5.5638859,⋯,4.4160681,4.6658248,3.5892766,3.557216,8.0141911,3.3057297,4.3123775,4.7737479,4.071993,5.3872617
ENSG00000000938,531.191987,614.806153,486.8702248,643.5142218,556.242408,649.4783794,561.8686380,465.7074293,640.7090237,558.3422915,⋯,584.7271866,648.2493407,636.1081486,610.561735,467.9885487,565.5175858,520.6040472,516.1287514,642.586687,538.0614569


In [45]:
names(txi)

[1] "abundance"           "counts"              "length"             
[4] "countsFromAbundance"

In [52]:
?tximport

tximport {tximport},R Documentation
files,a character vector of filenames for the transcript-level abundances
type,"character, the type of software used to generate the abundances. Options are ""salmon"", ""sailfish"", ""alevin"", ""kallisto"", ""rsem"", ""stringtie"", or ""none"". This argument is used to autofill the arguments below (geneIdCol, etc.) ""none"" means that the user will specify these columns."
txIn,"logical, whether the incoming files are transcript level (default TRUE)"
txOut,"logical, whether the function should just output transcript-level (default FALSE)"
countsFromAbundance,"character, either ""no"" (default), ""scaledTPM"", ""lengthScaledTPM"", or ""dtuScaledTPM"". Whether to generate estimated counts using abundance estimates: scaled up to library size (scaledTPM), scaled using the average transcript length over samples and then the library size (lengthScaledTPM), or scaled using the median transcript length among isoforms of a gene, and then the library size (dtuScaledTPM). dtuScaledTPM is designed for DTU analysis in combination with txOut=TRUE, and it requires specifing a tx2gene data.frame. dtuScaledTPM works such that within a gene, values from all samples and all transcripts get scaled by the same fixed median transcript length. If using scaledTPM, lengthScaledTPM, or geneLengthScaledTPM, the counts are no longer correlated across samples with transcript length, and so the length offset matrix should not be used."
tx2gene,"a two-column data.frame linking transcript id (column 1) to gene id (column 2). the column names are not relevant, but this column order must be used. this argument is required for gene-level summarization, and the tximport vignette describes how to construct this data.frame (see Details below). An automated solution to avoid having to create tx2gene if one has quantified with Salmon or alevin with human or mouse transcriptomes is to use the tximeta function from the tximeta Bioconductor package."
varReduce,"whether to reduce per-sample inferential replicates information into a matrix of sample variances variance (default FALSE). alevin computes inferential variance by default for bootstrap inferential replicates, so this argument is ignored/not necessary"
dropInfReps,"whether to skip reading in inferential replicates (default FALSE). For alevin, tximport will still read in the inferential variance matrix if it exists"
infRepStat,"a function to re-compute counts and abundances from the inferential replicates, e.g. matrixStats::rowMedians to re-compute counts as the median of the inferential replicates. The order of operations is: first counts are re-computed, then abundances are re-computed. Following this, if countsFromAbundance is not ""no"", tximport will again re-compute counts from the re-computed abundances. infRepStat should operate on rows of a matrix. (default is NULL)"
ignoreTxVersion,"logical, whether to split the tx id on the '.' character to remove version information to facilitate matching with the tx id in tx2gene (default FALSE)"


In [44]:
# Get tx2g
tx2g <- data.table::fread(Sys.glob(glue::glue('/sc/hydra/projects/GuccioneLab/genome-indices/{genome}/ensembl/*-biomart.txt')))[,c(4, 1)]

# Import
txi <- tximport(infiles, type='kallisto', txOut=FALSE, tx2g=tx2g)


1 
2 
3 
4 
5 
6 
7 
8 
9 
10 
11 
12 
13 
14 
15 
16 
17 
18 
19 
20 
21 
22 
23 
24 


transcripts missing from tx2gene: 1419

summarizing abundance

summarizing counts

summarizing length



In [14]:
txi <- tximport(infiles, type='kallisto', txOut = TRUE)

Note: importing `abundance.h5` is typically faster than `abundance.tsv`

reading in files with read_tsv

1 
Warning message:
“Duplicated column names deduplicated: '1147098' => '1147098_1' [3], '1147098' => '1147098_2' [4]”
Warning message:
“Unnamed `col_types` should have the same length as `col_names`. Using smaller of the two.”


ERROR: Error in tximport(infiles, type = "kallisto", txOut = TRUE): all(c(abundanceCol, countsCol, lengthCol) %in% names(raw)) is not TRUE
